Please read the following instructions thoroughly. Neglecting to do so may result in missed points.

### Preamble
**Reminder**: Homeworks are due by 7:00PM ET on Sundays.

Before you turn this problem set in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

### Naming conventions
Be sure the filename of your notebook is in the following form:

    <uni>_<assignment>_<details [optional]>.<extension>
    
For example:

    lr3086_hw01.ipynb
    lr3086_hw01_complete.ipynb
    LR3086_HW01.ipynb
    
To rename a notebook, in the menubar, select File$\rightarrow$Rename. The extension for notebook files, `.ipynb`, will already be appended to the filename, but will be hidden from view within the notebook.
    
This naming format allows for autograding of all assignments. If your files are not named with this format, you should expect a grade of zero for the assignment.

Courseworks may rename your file to something like `lr3086_hw0-1.ipynb` if you resubmit your assignment. This is perfectly fine.

### What Format To Submit In

Most homeworks are in Jupyter notebooks. Once you've finished your homework, unless specified otherwise, please download your work as an `.ipynb` file to your local machine, then upload it to Courseworks when complete (in the menubar, select File$\rightarrow$Download as$\rightarrow$Notebook).

**Failure to submit a Jupyter notebook will result in a grade of zero for the assignment.**

### Grading

Possible points on late assignments are deducted by 50% for each day they are late. For example, if you get 80% of the total possible credits on a homework but hand in that homework a day late, you would get 40%. Assignments two days late get zero points.

Once solutions are posted and graded assignments are handed back, students have 1 week to bring their grading discrepancies to a CA for consideration of possible grading errors.

Because grading is automated, please delete (or comment out) the `raise NotImplmeneted` code before attempting a problem.

Empty un-editable cells in an assignment are there for a reason. They will be filled with tests by the automatic grader. Please do not attempt to remove them.

### Getting Help

Asking for help is a great way to increase your chance of success. However there are some rules. When asking for help (especially from a fellow student), *you can show your helper your code but you can not view theirs*. You work needs to be your own. You can not post screenshots of your current work to Ed Discussions or other tools used for getting help.

If you need to reach out to a CA for help, please do so via Ed Discussions and not via email. Answers given via Ed Discussions will help you as well as other students. Thus, emails will always have a lower priority for response than Ed Discussions questions. If you do email the CA, please make a note of what section you are in. This helps us identify you in Courseworks faster.

Finally, if you do not get a repsonse from a CA within 48 hours, you may email the professor.

---

# Homework 9: SQL

Total questions: 6<br/>
Total points: 11

### FYI

In Monday's lecture, we used paths on your local machine outside of the notebook to persist data. But for the homework, we use a special argument – `:memory:` – that `sqlite3` has which stores the database in-memory; in otherwords, the database will live _**only so long as you're running the notebook**_. Afterwards, it will be thrown away. Usually this argument is useful when experimenting, given that as we discussed, it is persistence that often makes databases useful. We use it here however simply to avoid needing to upload multiple files to Courseworks; everything you need is here, and it makes it far easier to grade.

If, for any reason, you need to discard the contents of your database to start over, you may therefore do so by simply restarting the "kernel" of this notebook, which you can do by clicking the restart button in the toolbar (it's 2 buttons to the right of the "Run" button) or in the Kernel menu.

In [21]:
# Execute this cell before starting your homework!
# Do not delete this cell. You will receive a 0 on this homework otherwise.

import sqlite3

# this is our in-memory database, not stored on your hard drive
connection = sqlite3.connect(":memory:")

## Question 1

We'll start by creating some tables to store our data -- we'll first have one containing some product information.

Our `products` table will have:

* a name (`TEXT`)
* a description (`TEXT`)
* a unit cost stored in cents (`INTEGER`)

and of course we also add an id column to identify them.

The SQL statement to create each table is below. Execute it on the connection provided below to create the table.

```sql
CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY,
    name TEXT,
    description TEXT,
    unit_cost_cents INTEGER
)
```

[0.5 points]

In [22]:
with connection:
    connection.execute(
        """
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY,
            name TEXT,
            description TEXT,
            unit_cost_cents INTEGER
        )
        """
    )

In [23]:
### BEGIN TESTS

# Querying SQLite to ensure the `products` table is created
with connection:
    result = connection.execute(
        """
        SELECT name FROM sqlite_master WHERE type='table';
        """
    )
    tables = {table for table, in result}
assert "products" in tables
### END TESTS

In [24]:
# CELL INTENTIONALLY LEFT BLANK - DO NOT ALTER OR DELETE

## Question 2

Separately, we'd like to track the number of items in stock for each product.

To do so, we'll have two more tables: `stores` and `inventory`.

Stores have just an `id` and a `name`. **Create this table** yourself with a similar `CREATE TABLE` statement to the above.

Then, our `inventory` table should combine stores and products, listing how much of each product each store has in stock.

* a `product_id` (`INTEGER`)
* a `store_id` (`INTEGER`)
* a `quantity` (`INTEGER`) in stock

_(Note: in this case below, we actually don't need a separate `id` column because we can use the combination of `store_id` and `product_id` to identify each row. This is known as a_ composite primary key _and is another fundamental relational database concept, but can be ignored if need be for our purposes)._

Here is a statement creating this table:

```sql
CREATE TABLE IF NOT EXISTS inventory (
    product_id INTEGER NOT NULL,
    store_id INTEGER NOT NULL,
    quantity INTEGER,
    FOREIGN KEY (product_id) REFERENCES products(id),
    FOREIGN KEY (store_id) REFERENCES stores(id),
    PRIMARY KEY (product_id, store_id)
)
```
which **you need to execute as well**.

[0.5 points]

In [25]:
with connection:
    # Create the 'stores' table
    connection.execute(
        """
        CREATE TABLE IF NOT EXISTS stores (
            id INTEGER PRIMARY KEY,
            name TEXT
        )
        """
    )

    # Create the 'inventory' table
    connection.execute(
        """
        CREATE TABLE IF NOT EXISTS inventory (
            product_id INTEGER NOT NULL,
            store_id INTEGER NOT NULL,
            quantity INTEGER,
            FOREIGN KEY (product_id) REFERENCES products(id),
            FOREIGN KEY (store_id) REFERENCES stores(id),
            PRIMARY KEY (product_id, store_id)
        )
        """
    )


In [26]:
# CELL INTENTIONALLY LEFT BLANK - DO NOT ALTER OR DELETE

In [27]:
# CELL INTENTIONALLY LEFT BLANK - DO NOT ALTER OR DELETE

## Question 3

Now we can insert some stores, products and inventory into our database.

There are 2 stores -- one called `NY` and one called `NJ`. There are 2 products we are concerned with. Their names are `sneakers`, costing `$220` (remember this is dollars!) and `boots` costing `$350`. Use any description for each that you'd like. `NY` has 4 `sneakers` in stock and 3 `boots`. `NJ` has 5 `sneakers` in stock and no `boots`.

Insert the above data into the tables you have created.

[4 points]

In [28]:
with connection:
    # Insert data into the 'stores' table
    connection.execute("INSERT INTO stores (name) VALUES ('NY'), ('NJ')")

    # Insert data into the 'products' table
    # Remember to convert the cost from dollars to cents
    connection.execute(
        """
        INSERT INTO products (name, description, unit_cost_cents) VALUES
        ('sneakers', 'A comfortable pair of sneakers', 22000),
        ('boots', 'Durable and stylish boots', 35000)
        """
    )

    # Insert data into the 'inventory' table
    # Assuming the IDs for NY and NJ are 1 and 2 respectively,
    # and the IDs for sneakers and boots are 1 and 2 respectively
    connection.execute(
        """
        INSERT INTO inventory (product_id, store_id, quantity) VALUES
        (1, 1, 4), -- 4 sneakers in NY
        (2, 1, 3), -- 3 boots in NY
        (1, 2, 5), -- 5 sneakers in NJ
        (2, 2, 0)  -- 0 boots in NJ
        """
    )


In [29]:
# CELL INTENTIONALLY LEFT BLANK - DO NOT ALTER OR DELETE

In [30]:
# CELL INTENTIONALLY LEFT BLANK - DO NOT ALTER OR DELETE

In [31]:
# CELL INTENTIONALLY LEFT BLANK - DO NOT ALTER OR DELETE

## Question 4

We can of course calculate the total amount of products across all stores.

Write, but do not execute! a `SELECT` query which calculates this sum and returns it as its single returned column. You'll want to use a `SQL` function called `SUM` which functions much like the `COUNT` aggregation function we used in class.

Assign your query to a variable called `query`.

[1.5 point]

In [32]:
query = """
SELECT SUM(quantity) AS total_products
FROM inventory
"""

In [33]:
### BEGIN TESTS

# Ensure variable exists
assert query

### END TESTS

In [34]:
# CELL INTENTIONALLY LEFT BLANK - DO NOT ALTER OR DELETE

In [35]:
# CELL INTENTIONALLY LEFT BLANK - DO NOT ALTER OR DELETE

## Question 5

What about the total value _in cents_ of all products in stock? I.e. the `sum of (quantity in stock * cost of product across all products)`?

Create a function called `total_current_value` which returns this `int`. You may calculate this entirely in SQL or partially in Python after retrieving the data you need from the database. You must however retrieve some of the data from the database.

[1.5 points]

In [36]:
def total_current_value():
    query = """
    SELECT SUM(i.quantity * p.unit_cost_cents) AS total_value
    FROM inventory i
    JOIN products p ON i.product_id = p.id
    """
    with connection:
        result = connection.execute(query).fetchone()
        return result[0] if result[0] is not None else 0

# Usage
actual = total_current_value()


In [37]:
### BEGIN TESTS
expected = 303_000
actual = total_current_value()

assert expected == actual, f"\nExpected: {expected}\nGot: {actual}"
### END TESTS

In [38]:
# CELL INTENTIONALLY LEFT BLANK - DO NOT ALTER OR DELETE

## Question 6

Let's say we have a spreadsheet of historical hourly weather information:

| timestamp           | station_code | temp  | conditions |
|---------------------|--------------|-------|------------|
| 2023-03-26 14:00:00 | 12           | 56.4  | Clear Sky  |
| 2023-03-26 13:30:00 | 11           | 42.0  | Few Clouds |
| 2023-03-26 13:00:00 | 12           | 55.8  | Clear Sky  |
| 2023-03-26 12:30:00 | 11           | 41.5  | Overcast   |


We want to put all that historical data into our database, but first: we have to create the table.

Write a string that, when executed, will create a table called `hourly_weather` for our weather information. Assign that string to the variable `weather_schema`. 

Be sure to:
* define & declare a primary key named `id`
* use the column titles in the example data as column names in your schema
* use appropriate data types for each of the columns
* write a valid SQL statement
* ensure that if the `CREATE` SQL statement is executed multiple times, it does not throw an exception

You can assume the data types deduced in the example will _not_ change. For example, the station code will always be an integer.

Refer to the first column of the table in section [3.1.1 Affinity Name Examples](https://www.sqlite.org/datatype3.html#affinity_name_examples) in SQLite's documentation for better names of data types to use other than the basics it supports. Specifically, any of the following: `INTEGER`, `FLOAT`, `REAL`, `BLOB`, `TEXT`, `BOOLEAN`, `DATE`, and `DATETIME`.

[3 points]

In [39]:
# Execute this cell before implementing your answer.
# Do not delete this cell. You will receive a 0 on this homework otherwise.

# make a new in-memory database
connection = sqlite3.connect(":memory:")

In [40]:
weather_schema = """
CREATE TABLE IF NOT EXISTS hourly_weather (
    id INTEGER PRIMARY KEY,
    timestamp DATETIME NOT NULL,
    station_code INTEGER NOT NULL,
    temp REAL NOT NULL,
    conditions TEXT NOT NULL
)
"""


In [41]:
### BEGIN TESTS

# Ensure variable is defined
assert weather_schema

# Ensure valid SQL is written
try:
    with connection:
        connection.execute(weather_schema)
except Exception as e:
    msg = f"Got the following error when executing SQL statement: {e}"
    assert False, msg

### END TESTS

In [42]:
### BEGIN TESTS

# Ensure the table created has the expected name
TABLE_NAME_QUERY = """
SELECT name 
FROM sqlite_master 
WHERE type='table' AND name NOT LIKE 'sqlite_%'
"""

with connection:
    table, = list(connection.execute(TABLE_NAME_QUERY))
    

expected = "hourly_weather"
actual = table[0]
assert expected == actual, f"\nExpected: {expected}\nGot: {actual}"
### END TESTS

In [43]:
# CELL INTENTIONALLY LEFT BLANK - DO NOT ALTER OR DELETE

In [44]:
### BEGIN TESTS

# Ensure expected column names
TABLE_NAME_QUERY = """
SELECT name 
FROM sqlite_master 
WHERE type='table' AND name NOT LIKE 'sqlite_%'
"""
with connection:
    table, = list(connection.execute(TABLE_NAME_QUERY))
    actual_table_name = table[0]
    
QUERY = f"PRAGMA table_info({actual_table_name})"

with connection:
    rows = list(connection.execute(QUERY))

expected_column_names = sorted(["id", "timestamp", "station_code", "temp", "conditions"])
actual_column_names = sorted([row[1] for row in rows])

assert expected_column_names == actual_column_names, f"Expected: {', '.join(expected_column_names)}\nGot: {', '.join(actual_column_names)}"
### END TESTS

In [45]:
# CELL INTENTIONALLY LEFT BLANK - DO NOT ALTER OR DELETE

In [46]:
# CELL INTENTIONALLY LEFT BLANK - DO NOT ALTER OR DELETE